# annotation

> This module contains code for the annotation tab of the application

In [1]:
#| default_exp annotation

In [2]:
#| hide
from nbdev.showdoc import *

These are the relevant library calls to build out the UI and the calls to the model in order to set up the manuscript selection buttons.

In [3]:
#| hide
# This library is only for testing the layout. It does not need to be called extraneously
import dash

In [4]:
#| export
from dash import dcc, html
import dash_bootstrap_components as dbc
import plotly.express as px
import cv2

## createAnnotationInfo

Creates an info pane

In [5]:
#| export
def createAnnotationInfo():
    return dbc.Card(
        dbc.CardBody(
            [
                html.H1("Annotation"),
                html.P(
                    "This page allows you to annotate a manuscript page with Lines and Boxes. "
                    "For each line of a manuscript, draw a line through the text. "
                    "For each word of a manuscript, draw a box around the word. "
                    "Once you have drawn all the shapes, click the Save Shapes button. "
                ),
                html.Br(),
                html.P(
                    "After the shapes have been saved, the text editing pane on the right will allow you to transcribe the manuscript with the annotations. "
                    "For each line you draw, numbers will appear in the text area. "
                    "Leaving a space after each line number, write the words corresponding to that line inside the area. "
                    "Once you have finished the transcription of a page, click the Save Transcription button. "
                    "You can now select a different page to annotate and transcribe."
                )
            ]
        )
    )

In [1]:
#| hide
app = dash.Dash(
    external_stylesheets=[dbc.themes.BOOTSTRAP]
)

app.layout = html.Div(
    createAnnotationInfo()
)

if __name__ == "__main__":
    app.run(debug=True)

NameError: name 'dash' is not defined

## createAnnotationTextArea

This creates a ```Texarea``` object. Without changing some of the JavaScript code, the ```Textarea``` object does not have the ability to display line numbers.

In [6]:
#| export
def createAnnotationTextArea():
    return dbc.Textarea(id="annotation-text-area",value="Enter transcription text here!",style={"width":"100%","height":285})

## createPageSelector

For any manuscript that is in use, it is not unexpected for there to be multiple pages. In the case that there are multiple pages, it would be good for users to be able to select them. To that end, this function creates a [```Dropdown```](https://dash.plotly.com/dash-core-components/dropdown) object. By the nature of this application, the ```options``` will have to be dynamically assigned during run time.

In [7]:
#| export
def createPageSelector():
    return dcc.Dropdown(id="page-selector")

## createSaveShapes

In my testing, I found that trying to keep the ```BBox``` and ```Line``` classes automatically saving to a ```csv``` was problematic. It used enough computational resources to be frustrating and required quick algorithms to parse a dictionary into the desired format that, by all accounts, would take a time to execute of searching the dictionary (so $D$ elements) times the amount of time to sort the lines ($L$ elements) times the amount of time to sort the bboxes on two axes ($B^2$), leading to just enough resources being used to take a little time and all of it being run through the Python-Dash-JS pipeline just made me worry that computers with lower specs would have a difficult time working through it.

This button is to be called in order to save the current shapes on the annotation figure.

In [8]:
#| export
def createSaveShapes():
    return dbc.Button("Save Shapes", color="info", id="save-shapes")

## createSaveAnnotation

Similar to the motivation to the ```createSaveShapes``` function, this is necessary in order to make sure to only take the state of the annotation figure and use it for adding the annotation to the BBox class before writing the classes to memory.

In [9]:
#| export
def createSaveAnnotation():
    return dbc.Button("Save Transcription", color="info", id="save-annotation")

## createNextTab

To keep users able to move on to the next tab without using the tab object, this button is created. It's not a complicated button.

In [10]:
#| export
def createNextTab():
    return dbc.Button("Next Tab", color="primary", id="next-tab")

## createAnnotationFigure

The heart of this program is right here. This is how boxes will be drawn. In order to do this, we will use the [```Imshow```](https://plotly.com/python/imshow/) chart from the ```Plotly``` library. There are [ways](https://dash.plotly.com/annotations) of linking this figure into the application. Fortunately or unfortunately, it is not possible to keep one plot object with an image inside it. Instead, the figure needs to be made again and again for each image fed into it. Then, this plot is wrapped in the ```Graph``` object from the ```Dash``` library to be given an ```id``` for callback features. This wrapping needs to be handled in the ```Dash``` app itself, not the view calls.

This function takes in a pathway to a manuscript image as a string and returns a ```Plotly``` figure.

Due to some quirks of the ```cv2``` library, using ```cv2.imread()``` will return an image with BGR ordering of color channels, not RGB ordering of color channels. Computer monitors use the RGB color scheme as a way of representing colors for display:

<img
     style = "display:block;
              margin-left:auto;
              margin-right: auto;
              width:20%;"
     src = "https://upload.wikimedia.org/wikipedia/commons/9/97/RGB_color_wheel_pixel_30.svg"
/>

In an LED monitor, every pixel is made of a red, blue, and green LED that will show different intensities of colored light. Combining these pixels together into one system allows us to display images, movies, videos, etc... by coordingating the display of colors for each LED in each pixel of a monitor. What does this have to do with images? In mathematical terms, we might say that an image $I$ will have scalar intensities associated to three different coordinates. The first are the traditional $(x,y)$ Cartesian coordinate structure, but the third coordinate is color. We might say then that an individual LED at some position on the monitor will display the value $I(x_0,y_0,c)$ at the LED (where $c$ is theoretically the associated color channel). Most monitors use RGB (where red is the first index, green is the second index, and green is the third index), but the ```cv2``` library uses the BGR scheme. It is essentially the same as the RGB scheme, but a computer monitor will display red as if it is blue and blue as if it is red, distorting the image. To fix this, this function automatically reverses the ordering of BGR to RGB, hence the slicing.

In [11]:
#| export
def createAnnotationFigure(path):
    img = cv2.imread(path)
    # This reorders the color channels (the first two indices relate to the intensity values of individual colors while the last index indicates what
    img = img[:,:,::-1]
    fig = px.imshow(img)
    fig.update_layout(dragmode='drawrect',
                  # style of new shapes
                  newshape=dict(line_color='grey',
                                opacity=0.6))
    return fig

## createFigureLayout

This creates the layout for the figure

In [12]:
#| export
def createFigureLayout():
    return dbc.Card(
        [
            createPageSelector(),
            dcc.Graph(
                id="annotation-figure",
                config={
                    "modeBarButtonsToAdd": [
                        "drawline",
                        "drawrect",
                        "eraseshape",
                    ]
                },
            ),
            createSaveShapes(),
        ]
    )

## createTextareaLayout

This creates the layout for the textarea

In [13]:
#| export
def createTextareaLayout():
    return dbc.Card([createAnnotationTextArea(), createSaveAnnotation()])

## createAnnotationLayout

Does what it says on the tin

In [21]:
#| export
def createAnnotationLayout():
    return html.Div(
        [
            createAnnotationInfo(),
            html.Br(),
            dbc.Card(
                [
                    dbc.Container(
                        dbc.Row(
                            children=[
                                dbc.Col(createFigureLayout(), md=7),
                                dbc.Col(createTextareaLayout(), md=5),
                            ]
                        )
                    ),
                    createNextTab(),
                ]
            ),
        ]
    )

In [22]:
#| hide
app = dash.Dash(
    external_stylesheets=[dbc.themes.BOOTSTRAP]
)
app.layout = createAnnotationLayout()

if __name__ == "__main__":
    app.run()

In [16]:
#| hide
import nbdev

nbdev.nbdev_export()